<a href="https://colab.research.google.com/github/bellDataSc/Projeto-ETL-com-Python-e-Google-BigQuery/blob/main/brazilian_demographics_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brazilian Demographics ETL Pipeline

**Extract, Transform, Load & Analyze Brazilian Population Data from IBGE APIs**

This notebook demonstrates a comprehensive ETL pipeline using Brazilian government data sources, featuring:
- Data extraction from multiple IBGE APIs
- Data transformation and cleaning
- Interactive visualizations and demographic analysis
- Data export capabilities

**Author:** Isabel Cruz

**Data Source:** Brazilian Institute of Geography and Statistics (IBGE)  
**Last Updated:** August 2025

## Environment Setup

In [1]:

!pip install requests pandas matplotlib seaborn plotly geopandas -q
!pip install google-cloud-bigquery python-dotenv -q

In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline

print("Environment setup complete")

Environment setup complete


## Data Extraction

In [3]:
def extract_states_data():
    """Extract Brazilian states data from IBGE API"""
    try:
        url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados"
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        print(f"Successfully extracted {len(data)} states")
        return data
    except Exception as e:
        print(f"Error extracting states data: {e}")
        return None


states_raw = extract_states_data()

Successfully extracted 27 states


In [4]:
def extract_municipalities_data(limit_states=5):
    """Extract municipalities data for selected states"""
    municipalities_data = []


    top_states = ['SP', 'MG', 'RJ', 'BA', 'PR'][:limit_states]

    for state in top_states:
        try:
            url = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{state}/municipios"
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            data = response.json()
            municipalities_data.extend(data)
            print(f"Extracted {len(data)} municipalities from {state}")
        except Exception as e:
            print(f"Error extracting data for {state}: {e}")

    return municipalities_data


municipalities_raw = extract_municipalities_data()

Extracted 645 municipalities from SP
Extracted 853 municipalities from MG
Extracted 92 municipalities from RJ
Extracted 417 municipalities from BA
Extracted 399 municipalities from PR


In [5]:
def extract_regions_data():
    """Extract Brazilian regions data"""
    try:
        url = "https://servicodados.ibge.gov.br/api/v1/localidades/regioes"
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        print(f"Successfully extracted {len(data)} regions")
        return data
    except Exception as e:
        print(f"Error extracting regions data: {e}")
        return None


regions_raw = extract_regions_data()

Successfully extracted 5 regions


## Data Transformation

In [7]:
def transform_states_data(raw_data):
    """Transform states data into clean DataFrame"""
    if not raw_data:
        return None

    df = pd.json_normalize(raw_data)


    df = df.rename(columns={
        'nome': 'state_name',
        'sigla': 'state_code',
        'regiao.nome': 'region_name',
        'regiao.sigla': 'region_code',
        'regiao.id': 'region_id'
    })


    df['extraction_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df['state_name_length'] = df['state_name'].str.len()

    print(f"Transformed states data: {df.shape[0]} rows × {df.shape[1]} columns")
    return df


states_df = transform_states_data(states_raw)
if states_df is not None:
    display(states_df.head())

Transformed states data: 27 rows × 8 columns


,id,state_code,state_name,region_id,region_code,region_name,extraction_date,state_name_length
0,11,RO,Rondônia,1,N,Norte,2025-08-25 21:06:34,8
1,12,AC,Acre,1,N,Norte,2025-08-25 21:06:34,4
2,13,AM,Amazonas,1,N,Norte,2025-08-25 21:06:34,8
3,14,RR,Roraima,1,N,Norte,2025-08-25 21:06:34,7
4,15,PA,Pará,1,N,Norte,2025-08-25 21:06:34,4


In [8]:
def transform_municipalities_data(raw_data):
    """Transform municipalities data into clean DataFrame"""
    if not raw_data:
        return None

    df = pd.json_normalize(raw_data)


    df = df.rename(columns={
        'nome': 'municipality_name',
        'microrregiao.nome': 'microregion_name',
        'microrregiao.mesorregiao.nome': 'mesoregion_name',
        'microrregiao.mesorregiao.UF.nome': 'state_name',
        'microrregiao.mesorregiao.UF.sigla': 'state_code',
        'microrregiao.mesorregiao.UF.regiao.nome': 'region_name'
    })


    df['extraction_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df['municipality_name_length'] = df['municipality_name'].str.len()

    print(f"📊 Transformed municipalities data: {df.shape[0]} rows × {df.shape[1]} columns")
    return df


municipalities_df = transform_municipalities_data(municipalities_raw)
if municipalities_df is not None:
    display(municipalities_df.head())

📊 Transformed municipalities data: 2406 rows × 24 columns


,id,municipality_name,microrregiao.id,microregion_name,microrregiao.mesorregiao.id,mesoregion_name,microrregiao.mesorregiao.UF.id,state_code,state_name,microrregiao.mesorregiao.UF.regiao.id,...,regiao-imediata.regiao-intermediaria.id,regiao-imediata.regiao-intermediaria.nome,regiao-imediata.regiao-intermediaria.UF.id,regiao-imediata.regiao-intermediaria.UF.sigla,regiao-imediata.regiao-intermediaria.UF.nome,regiao-imediata.regiao-intermediaria.UF.regiao.id,regiao-imediata.regiao-intermediaria.UF.regiao.sigla,regiao-imediata.regiao-intermediaria.UF.regiao.nome,extraction_date,municipality_name_length
0,3500105,Adamantina,35035,Adamantina,3508,Presidente Prudente,35,SP,São Paulo,3,...,3505,Presidente Prudente,35,SP,São Paulo,3,SE,Sudeste,2025-08-25 21:07:10,10
1,3500204,Adolfo,35004,São José do Rio Preto,3501,São José do Rio Preto,35,SP,São Paulo,3,...,3507,São José do Rio Preto,35,SP,São Paulo,3,SE,Sudeste,2025-08-25 21:07:10,6
2,3500303,Aguaí,35029,Pirassununga,3507,Campinas,35,SP,São Paulo,3,...,3510,Campinas,35,SP,São Paulo,3,SE,Sudeste,2025-08-25 21:07:10,5
3,3500402,Águas da Prata,35030,São João da Boa Vista,3507,Campinas,35,SP,São Paulo,3,...,3510,Campinas,35,SP,São Paulo,3,SE,Sudeste,2025-08-25 21:07:10,14
4,3500501,Águas de Lindóia,35033,Amparo,3507,Campinas,35,SP,São Paulo,3,...,3510,Campinas,35,SP,São Paulo,3,SE,Sudeste,2025-08-25 21:07:10,16


## Exploratory Data Analysis

In [10]:

print("DATASET OVERVIEW")
print("=" * 50)

if states_df is not None:
    print(f" States: {len(states_df)} total")
    print(f" Regions: {states_df['region_name'].nunique()} unique regions")

if municipalities_df is not None:
    print(f" Municipalities: {len(municipalities_df)} total")
    print(f" States covered: {municipalities_df['state_name'].nunique()}")

print(f" Data extracted on: {datetime.now().strftime('%Y-%m-%d at %H:%M:%S')}")

DATASET OVERVIEW
 States: 27 total
 Regions: 5 unique regions
 Municipalities: 2406 total
 States covered: 5
 Data extracted on: 2025-08-25 at 21:08:40


## Interactive Visualizations

In [11]:

if states_df is not None:

    region_counts = states_df['region_name'].value_counts()


    fig = px.pie(
        values=region_counts.values,
        names=region_counts.index,
        title=" Distribution of Brazilian States by Region",
        color_discrete_sequence=px.colors.qualitative.Set3
    )

    fig.update_traces(
        textposition='inside',
        textinfo='percent+label',
        hovertemplate='<b>%{label}</b><br>States: %{value}<br>Percentage: %{percent}<extra></extra>'
    )

    fig.update_layout(
        font_size=12,
        showlegend=True,
        height=500
    )

    fig.show()

In [13]:

if municipalities_df is not None:

    state_counts = municipalities_df['state_name'].value_counts()


    fig = px.bar(
        x=state_counts.index,
        y=state_counts.values,
        title=" Number of Municipalities by State (Sample)",
        labels={'x': 'State', 'y': 'Number of Municipalities'},
        color=state_counts.values,
        color_continuous_scale='Viridis'
    )

    fig.update_traces(
        hovertemplate='<b>%{x}</b><br>Municipalities: %{y}<extra></extra>'
    )

    fig.update_layout(
        showlegend=False,
        height=400,
        xaxis_title="State",
        yaxis_title="Number of Municipalities"
    )

    fig.show()

In [15]:

if municipalities_df is not None:

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Municipality Name Lengths Distribution', 'Name Lengths by State'),
        specs=[[{'type': 'histogram'}, {'type': 'box'}]]
    )


    fig.add_trace(
        go.Histogram(
            x=municipalities_df['municipality_name_length'],
            nbinsx=20,
            name='Distribution',
            marker_color='skyblue'
        ),
        row=1, col=1
    )


    for state in municipalities_df['state_code'].unique():
        state_data = municipalities_df[municipalities_df['state_code'] == state]
        fig.add_trace(
            go.Box(
                y=state_data['municipality_name_length'],
                name=state,
                boxpoints='outliers'
            ),
            row=1, col=2
        )

    fig.update_layout(
        title_text=" Municipality Names Analysis",
        showlegend=False,
        height=500
    )

    fig.update_xaxes(title_text="Name Length (characters)", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_xaxes(title_text="State", row=1, col=2)
    fig.update_yaxes(title_text="Name Length (characters)", row=1, col=2)

    fig.show()

## Data Summary & Insights

In [16]:

print(" KEY INSIGHTS")
print("=" * 40)

if states_df is not None:
    print(f" Brazil has {len(states_df)} states distributed across {states_df['region_name'].nunique()} major regions")


    most_common_region = states_df['region_name'].mode().iloc[0]
    region_state_count = states_df[states_df['region_name'] == most_common_region].shape[0]
    print(f" {most_common_region} region has the most states ({region_state_count} states)")

if municipalities_df is not None:
    print(f" Sample includes {len(municipalities_df)} municipalities from {municipalities_df['state_name'].nunique()} states")


    state_with_most = municipalities_df['state_name'].value_counts().index[0]
    municipality_count = municipalities_df['state_name'].value_counts().iloc[0]
    print(f" {state_with_most} has the most municipalities in our sample ({municipality_count} municipalities)")


    avg_name_length = municipalities_df['municipality_name_length'].mean()
    print(f" Average municipality name length: {avg_name_length:.1f} characters")


    longest_name_idx = municipalities_df['municipality_name_length'].idxmax()
    longest_name = municipalities_df.loc[longest_name_idx, 'municipality_name']
    longest_length = municipalities_df.loc[longest_name_idx, 'municipality_name_length']
    print(f" Longest municipality name: '{longest_name}' ({longest_length} characters)")

 KEY INSIGHTS
 Brazil has 27 states distributed across 5 major regions
 Nordeste region has the most states (9 states)
 Sample includes 2406 municipalities from 5 states
 Minas Gerais has the most municipalities in our sample (853 municipalities)
 Average municipality name length: 11.3 characters
 Longest municipality name: 'São Sebastião da Vargem Alegre' (30 characters)




##  Data Export

In [17]:
# Export cleaned data to CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

if states_df is not None:
    states_filename = f'brazilian_states_{timestamp}.csv'
    states_df.to_csv(states_filename, index=False)
    print(f" States data exported to: {states_filename}")

if municipalities_df is not None:
    municipalities_filename = f'brazilian_municipalities_{timestamp}.csv'
    municipalities_df.to_csv(municipalities_filename, index=False)
    print(f" Municipalities data exported to: {municipalities_filename}")

print(f"\n Files are ready for download from the Colab file browser!")


try:
    from google.colab import files

    download_choice = input("\nDownload files now? (y/n): ")
    if download_choice.lower() == 'y':
        if states_df is not None:
            files.download(states_filename)
        if municipalities_df is not None:
            files.download(municipalities_filename)
        print(" Downloads initiated!")
except ImportError:
    print(" To download files, use the file browser in the left sidebar")

 States data exported to: brazilian_states_20250825_212207.csv
 Municipalities data exported to: brazilian_municipalities_20250825_212207.csv

 Files are ready for download from the Colab file browser!

Download files now? (y/n): y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Downloads initiated!


In [20]:
# Uncomment and configure for BigQuery upload

# from google.cloud import bigquery

# # Configuration (update with your values)
# PROJECT_ID = "your-project-id"
# DATASET_ID = "brazilian_data"

# def upload_to_bigquery(df, table_name, project_id, dataset_id):
#     """Upload DataFrame to BigQuery"""
#     try:
#         client = bigquery.Client(project=project_id)
#         table_id = f"{project_id}.{dataset_id}.{table_name}"

#         job_config = bigquery.LoadJobConfig(
#             write_disposition="WRITE_TRUNCATE"  # Overwrite table
#         )

#         job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
#         job.result()  # Wait for completion

#         print(f" Uploaded {len(df)} rows to {table_id}")
#     except Exception as e:
#         print(f" Error uploading to BigQuery: {e}")

# Upload data (uncomment to use)
# if states_df is not None:
#     upload_to_bigquery(states_df, "states", PROJECT_ID, DATASET_ID)
# if municipalities_df is not None:
#     upload_to_bigquery(municipalities_df, "municipalities", PROJECT_ID, DATASET_ID)

# print(" BigQuery upload section is commented out. Configure credentials to use.")
# print(" Learn more: https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries#client-libraries-install-python")

## Conclusion

This ETL pipeline successfully demonstrates:

 **Data Extraction**: Retrieved data from multiple IBGE API endpoints  
 **Data Transformation**: Cleaned and normalized JSON data into structured DataFrames  
 **Data Analysis**: Generated insights about Brazilian geographic distribution  
 **Data Visualization**: Created interactive charts for better understanding  
 **Data Export**: Saved processed data in CSV format for further use  

### Next Steps
- Add more demographic data sources (population, economic indicators)
- Implement automated scheduling for data updates
- Create geographic visualizations with maps
- Build predictive models using the collected data

### Useful Links
- [IBGE API Documentation](https://servicodados.ibge.gov.br/api/docs/)
- [Brazilian Geographic Data](https://www.ibge.gov.br/en/)
- [Plotly Documentation](https://plotly.com/python/)

---

**Made with ☕ by Isabel Cruz | in Google Colab | Data from IBGE**